In [4]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import torch
import torch.nn as nn
import onnx
import onnxruntime as ort
import numpy as np

os.makedirs("../app/models", exist_ok=True)

PATH_TO_WEIGHTS = "../outputs/yolo/best_eye_detector.pth"
ONNX_PATH = "../app/models/yolo_eye_detector.onnx"

print(f"PyTorch version: {torch.__version__}")
print(f"ONNX version: {onnx.__version__}")
print(f"ONNX Runtime version: {ort.__version__}")

PyTorch version: 2.5.1
ONNX version: 1.19.1
ONNX Runtime version: 1.22.2


In [5]:
class YOLOv2EyeDetector(nn.Module):    
    def __init__(self, grid_size=7, num_anchors=3, num_classes=1):
        super(YOLOv2EyeDetector, self).__init__()
        self.grid_size = grid_size
        self.num_anchors = num_anchors
        self.num_classes = num_classes
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            
            nn.Conv2d(128, 128, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(256, 256, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            
            nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.1),
        )
        
        self.detection = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, num_anchors * (5 + num_classes), kernel_size=1, stride=1, padding=0),
        )
        
    def forward(self, x):
        x = self.features(x)
        
        x = self.detection(x)
        
        batch_size = x.shape[0]
        x = x.permute(0, 2, 3, 1) 
        x = x.reshape(batch_size, self.grid_size, self.grid_size, -1)
        
        return x

print("Model architecture defined")

Model architecture defined


In [6]:
device = torch.device("cpu")
model = YOLOv2EyeDetector().to(device)

print(f"Loading weights from: {PATH_TO_WEIGHTS}")

if not os.path.exists(PATH_TO_WEIGHTS):
    raise FileNotFoundError(f"Model file not found: {PATH_TO_WEIGHTS}")

try:
    state_dict = torch.load(PATH_TO_WEIGHTS, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()
    print("Detector Weights Loaded!")
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")
    
except Exception as e:
    print(f"Error loading: {e}")
    raise

Loading weights from: ../outputs/yolo/best_eye_detector.pth
Detector Weights Loaded!
Total parameters: 13,464,914


C:\Users\GN\AppData\Local\Temp\ipykernel_41016\2012846821.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(PATH_TO_WEIGHTS, map_location=device)


In [7]:
print("Exporting to ONNX...")

input_size = 224
dummy_input = torch.randn(1, 1, input_size, input_size, device=device)

print(f"Exporting with input size: {input_size}x{input_size}...")

with torch.no_grad():
    torch.onnx.export(
        model,
        dummy_input,
        ONNX_PATH,
        export_params=True,
        opset_version=13,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )

file_size = os.path.getsize(ONNX_PATH) / (1024 * 1024)
print(f"ONNX model exported: {ONNX_PATH}")
print(f"File size: {file_size:.2f} MB")

Exporting to ONNX...
Exporting with input size: 224x224...
ONNX model exported: ../app/models/yolo_eye_detector.onnx
File size: 51.34 MB


In [8]:
print("Verifying ONNX model...")

try:
    onnx_model = onnx.load(ONNX_PATH)
    onnx.checker.check_model(onnx_model)
    print("ONNX model valid")
    
    print(f"\nModel Info:")
    print(f"IR Version: {onnx_model.ir_version}")
    print(f"Producer: {onnx_model.producer_name}")
    print(f"Opset Version: {onnx_model.opset_import[0].version}")
    
    print(f"\nInput:")
    for input_tensor in onnx_model.graph.input:
        print(f"Name: {input_tensor.name}")
        shape = [dim.dim_value if dim.dim_value > 0 else 'dynamic' for dim in input_tensor.type.tensor_type.shape.dim]
        print(f"Shape: {shape}")
    
    print(f"\nOutput:")
    for output_tensor in onnx_model.graph.output:
        print(f"Name: {output_tensor.name}")
        shape = [dim.dim_value if dim.dim_value > 0 else 'dynamic' for dim in output_tensor.type.tensor_type.shape.dim]
        print(f"Shape: {shape}")
        
except Exception as e:
    print(f"ONNX model failed: {e}")
    raise

Verifying ONNX model...
ONNX model valid

Model Info:
IR Version: 7
Producer: pytorch
Opset Version: 13

Input:
Name: input
Shape: ['dynamic', 1, 224, 224]

Output:
Name: output
Shape: ['dynamic', 7, 7, 72]


In [9]:
print("Testing ONNX inference...")

ort_session = ort.InferenceSession(ONNX_PATH)

test_input = torch.randn(1, 1, 224, 224, device=device)

with torch.no_grad():
    pytorch_output = model(test_input).cpu().numpy()

onnx_output = ort_session.run(
    None,
    {"input": test_input.cpu().numpy()}
)[0]

diff = np.abs(pytorch_output - onnx_output).max()
print(f"\nMax difference between PyTorch and ONNX: {diff:.6f}")

if diff < 1e-4:
    print("ONNX model matches PyTorch model")
else:
    print("Outputs differ significantly")

print(f"\nPyTorch output shape: {pytorch_output.shape}")
print(f"ONNX output shape: {onnx_output.shape}")

Testing ONNX inference...

Max difference between PyTorch and ONNX: 0.000092
ONNX model matches PyTorch model

PyTorch output shape: (1, 7, 7, 72)
ONNX output shape: (1, 7, 7, 72)


In [10]:
import time

print("Running performance benchmark...")

num_iterations = 100
test_input_np = test_input.cpu().numpy()

model.eval()
start_time = time.time()
with torch.no_grad():
    for _ in range(num_iterations):
        _ = model(test_input)
pytorch_time = (time.time() - start_time) / num_iterations * 1000

ort_session_std = ort.InferenceSession(ONNX_PATH)
start_time = time.time()
for _ in range(num_iterations):
    _ = ort_session_std.run(None, {"input": test_input_np})
onnx_time = (time.time() - start_time) / num_iterations * 1000

print(f"\n{'='*50}")
print(f"INFERENCE PERFORMANCE (Average over {num_iterations} runs)")
print(f"{'='*50}")
print(f"PyTorch Model:       {pytorch_time:.2f} ms")
print(f"ONNX Standard:       {onnx_time:.2f} ms ({pytorch_time/onnx_time:.2f}x)")
print(f"{'='*50}")

Running performance benchmark...

INFERENCE PERFORMANCE (Average over 100 runs)
PyTorch Model:       26.14 ms
ONNX Standard:       12.10 ms (2.16x)
